# VILLESFR keys

In [1]:
import pandas as pd
VILLESFR = pd.read_json('/home/antoine/Documents/GitHub/toponym-disambiguation/VILLESFR/with_moredatas_826frenchcities_100124.json')

In [2]:
import re

# get tuple (1,1065) from string like "volume01-1065"
def get_tuple_from_string(string):
    pattern = r'(\d+)'
    numbers = re.findall(pattern, string)
    numbers = [int(num) for num in numbers]
    extracted_tuple = tuple(numbers)

    return extracted_tuple
                 
VILLESFR_keys = list(VILLESFR['key'].apply(get_tuple_from_string))
                 

In [3]:
len(VILLESFR_keys)

826

# DocBins Exploration

* Load the GeoEDdA spancat model annotations of the EDdA and LGE corpus and count spans



In [4]:
import os
import pandas as pd
import spacy
from spacy.tokens import DocBin, Doc

import matplotlib.pyplot as plt
import seaborn as sns


## Read the serialized bytes from the file

In [5]:
nlp = spacy.load("models/fr_spacy_custom_spancat_edda")
Doc.set_extension("metadata", default={}, force=True)

with open("edda_serialized_docs_240131.bin", "rb") as f:
    serialized_bytes = f.read()

doc_bin = DocBin().from_bytes(serialized_bytes)
docs = list(doc_bin.get_docs(nlp.vocab))
jsonl = []
for doc in docs:
    jsonl.append(dict(doc._.metadata, **doc.to_json()))

In [6]:
arcos = jsonl[10500]
arcos

{'work': 'EDdA',
 'volume': 1,
 'article': 3740,
 'paragraph': 1,
 'headword': 'ARCOS',
 'authors': 'unsigned',
 'domains': ':Géographie:',
 'text': "ARCOS, (Géog.) ville d'Espagne, dans l'Andalousie, sur un roc au pié duquel coule la riviere de Guadalette. Long. 12. 20. lat. 36. 40.",
 'tokens': [{'id': 0, 'start': 0, 'end': 5},
  {'id': 1, 'start': 5, 'end': 6},
  {'id': 2, 'start': 7, 'end': 8},
  {'id': 3, 'start': 8, 'end': 12},
  {'id': 4, 'start': 12, 'end': 13},
  {'id': 5, 'start': 13, 'end': 14},
  {'id': 6, 'start': 15, 'end': 20},
  {'id': 7, 'start': 21, 'end': 23},
  {'id': 8, 'start': 23, 'end': 30},
  {'id': 9, 'start': 30, 'end': 31},
  {'id': 10, 'start': 32, 'end': 36},
  {'id': 11, 'start': 37, 'end': 39},
  {'id': 12, 'start': 39, 'end': 49},
  {'id': 13, 'start': 49, 'end': 50},
  {'id': 14, 'start': 51, 'end': 54},
  {'id': 15, 'start': 55, 'end': 57},
  {'id': 16, 'start': 58, 'end': 61},
  {'id': 17, 'start': 62, 'end': 64},
  {'id': 18, 'start': 65, 'end': 68}

In [1]:
def print_json_architecture(json_data, indent=0):
    for key, value in json_data.items():
        print(" '-- " * indent + key)  # Print the key with appropriate indentation
        if isinstance(value, dict):
            print_json_architecture(value, indent + 1)  # Recursively call the function for nested dictionaries
        if isinstance(value, list):
            print_json_architecture(value[0], indent + 1)  # Recursively call the function for nested dictionaries

print_json_architecture(arcos)

NameError: name 'arcos' is not defined

In [8]:
# print textspan + label
for span in arcos["spans"]['sc']:
    print(arcos["text"][span["start"]:span["end"]], span["label"])


ville NC-Spatial
Espagne NP-Spatial
dans Relation
Guadalette NP-Spatial
Géog. Domain-mark
l'Andalousie NP-Spatial
la riviere NC-Spatial
ville d'Espagne ENE-Spatial
la riviere de Guadalette ENE-Spatial
ville d'Espagne, dans l'Andalousie ENE-Spatial


## VILLESFR spacy document collection

In [9]:
VILLESFR_collec = []
for doc in jsonl :
    if (doc['volume'], doc['article']) in VILLESFR_keys:
        VILLESFR_collec.append(doc)

VILLESFR_collec[11]

{'work': 'EDdA',
 'volume': 1,
 'article': 1369,
 'paragraph': 1,
 'headword': 'AIGURANDE',
 'authors': 'unsigned',
 'domains': ':Géographie:',
 'text': 'AIGURANDE, (Géog.) ville de France dans la Marche sur les confins du Berry. Long. 19. 35. lat. 46. 25.',
 'tokens': [{'id': 0, 'start': 0, 'end': 9},
  {'id': 1, 'start': 9, 'end': 10},
  {'id': 2, 'start': 11, 'end': 12},
  {'id': 3, 'start': 12, 'end': 16},
  {'id': 4, 'start': 16, 'end': 17},
  {'id': 5, 'start': 17, 'end': 18},
  {'id': 6, 'start': 19, 'end': 24},
  {'id': 7, 'start': 25, 'end': 27},
  {'id': 8, 'start': 28, 'end': 34},
  {'id': 9, 'start': 35, 'end': 39},
  {'id': 10, 'start': 40, 'end': 42},
  {'id': 11, 'start': 43, 'end': 49},
  {'id': 12, 'start': 50, 'end': 53},
  {'id': 13, 'start': 54, 'end': 57},
  {'id': 14, 'start': 58, 'end': 65},
  {'id': 15, 'start': 66, 'end': 68},
  {'id': 16, 'start': 69, 'end': 74},
  {'id': 17, 'start': 74, 'end': 75},
  {'id': 18, 'start': 76, 'end': 80},
  {'id': 19, 'start': 

## creating the opentapioca-doc

In [10]:
# Search for spans in doc and append to the value associated with (volume, article)
for doc in VILLESFR_collec:
    doc['NP_list'] = []
    for span in doc['spans']['sc']:
        if span['label'] == "NP-Spatial":
            npspatial = doc["text"][span["start"]:span["end"]]
            doc['NP_list'].append(npspatial) 

In [11]:
VILLESFR_collec[11]

{'work': 'EDdA',
 'volume': 1,
 'article': 1369,
 'paragraph': 1,
 'headword': 'AIGURANDE',
 'authors': 'unsigned',
 'domains': ':Géographie:',
 'text': 'AIGURANDE, (Géog.) ville de France dans la Marche sur les confins du Berry. Long. 19. 35. lat. 46. 25.',
 'tokens': [{'id': 0, 'start': 0, 'end': 9},
  {'id': 1, 'start': 9, 'end': 10},
  {'id': 2, 'start': 11, 'end': 12},
  {'id': 3, 'start': 12, 'end': 16},
  {'id': 4, 'start': 16, 'end': 17},
  {'id': 5, 'start': 17, 'end': 18},
  {'id': 6, 'start': 19, 'end': 24},
  {'id': 7, 'start': 25, 'end': 27},
  {'id': 8, 'start': 28, 'end': 34},
  {'id': 9, 'start': 35, 'end': 39},
  {'id': 10, 'start': 40, 'end': 42},
  {'id': 11, 'start': 43, 'end': 49},
  {'id': 12, 'start': 50, 'end': 53},
  {'id': 13, 'start': 54, 'end': 57},
  {'id': 14, 'start': 58, 'end': 65},
  {'id': 15, 'start': 66, 'end': 68},
  {'id': 16, 'start': 69, 'end': 74},
  {'id': 17, 'start': 74, 'end': 75},
  {'id': 18, 'start': 76, 'end': 80},
  {'id': 19, 'start': 

In [2]:
print_json_architecture(VILLESFR_collec[11])

NameError: name 'VILLESFR_collec' is not defined

In [21]:
processed_articles= []

for idx, row in VILLESFR.iterrows():
    volume, article = get_tuple_from_string(row['key'])

    OpenTapiocaDoc = ""

    if (volume, article) not in processed_articles:
        if row.get('SAINT'):
            OpenTapiocaDoc += "SAINT "
        
        OpenTapiocaDoc += row['head'].upper() 
    
    paragraphs = [doc['NP_list'] for doc in VILLESFR_collec if (doc['volume'], doc['article']) == (volume, article)]
    print(paragraphs)
    
    for para in paragraphs:
        OpenTapiocaDoc += " % "
        OpenTapiocaDoc += " % ".join(para)
    
    VILLESFR.at[idx, 'OpenTapiocaDoc'] = OpenTapiocaDoc

    processed_articles.append((volume, article))

[['France', 'Gascogne', 'Montauban']]
[['France', 'Languedoc', 'Paris', "d'Agadez"]]
[['France', "l'Agénois", 'la Guienne', 'la Garonne']]
[['France', 'Monts', 'le haut-Vivarez']]
[['France', 'Rouen', "d'Evreux", 'la haute Normandie']]
[['France', 'Bourgogne', 'Dijon']]
[['France', 'le Berry', 'le Cher']]
[['France', 'Bourgogne', 'Dijon']]
[['France', 'le bas Languedoc']]
[['France', 'la basse Auvergne']]
[['France', 'Guyenne', "l'Agenois"]]
[['France', 'Berry', 'la Marche']]
[['France', 'la Gascogne', "l'Adour"]]
[['France', 'Bourgogne', 'Chatillon']]
[['France', 'Bourges']]
[['France', 'Provence', "d'Arc"]]
[['France', 'le Limosin', 'la Marche']]
[['France', 'Gardon', 'le bas Languedoc']]
[['France', 'Mende', 'le bas Languedoc']]
[['France', "l'Albigeois", 'le Tarn', 'le haut Languedoc']]
[['France', 'Gascogne', "d'Albret"]]
[['France', 'la Sarte', 'la basse Normandie'], ['Alençon', 'Alençon', 'Bernay', 'Bernay', 'Lizieux', 'Brionne', 'Domfront', 'Vimoutiers', 'France', 'Alençon', 'L

In [22]:
VILLESFR[['key', 'OpenTapiocaDoc']]

,key,OpenTapiocaDoc
0,volume01-1065,AFRIQUE % France % Gascogne % Montauban
1,volume01-1087,AGDE % France % Languedoc % Paris % d'Agadez
2,volume01-1103,AGEN % France % l'Agénois % la Guienne % la Ga...
3,volume01-1210,AGRERE % France % Monts % le haut-Vivarez
4,volume01-1279,AIGLE % France % Rouen % d'Evreux % la haute N...
...,...,...
871,volume17-2393,WORDT % France % Hanau % Wasgaw % la basse-Als...
872,volume17-2434,XAINTES % France % la Saintonge
873,volume17-2619,"SAINT YRIER DE LA PERCHE, SAINT % France % le ..."
874,volume17-2632,YVETOT % France % Normandie % Caux % Caudebec ...


In [23]:
VILLESFR.to_json('VILLESFR_with_OpenTapiocaDoc.json')

In [32]:
df = pd.DataFrame(VILLESFR_collec)
df.to_json('VILLESFR_collec.json')